In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.stats as stats
from scipy.optimize import fmin

In [82]:
lambda_1 = 0.2
T = 5
lambda_2 = 0.4
delta = 0.25
r0 = 0.055
R = r0 + delta * lambda_2
L0 = 22_000
r = R/12
n = T*12
dt = T/n
mu = -0.1
sigma = 0.2
gamma = -0.4
V0 = 20_000
alpha = 0.7
epsilon = 0.95
beta = (epsilon - alpha)/T
PMT = (L0 * r) /(1-(1/(1+r)**n))
a = PMT/r
b = PMT/(r * (1+r)**n)
c = (1+r)
M = 100_00
t = np.linspace(0, T, n + 1)
# qT = epsilon embedded in derivation of qt
qt = alpha + beta * t
Lt = a - b * c **(12 * t)
payoff_final = np.zeros(10_000)
tao_list = np.zeros(10_000)
for i in range(M):
    Y = [np.random.exponential(scale=1/(lambda_1*T), size=1)[0]]
    while Y[-1] < T:
        Y_new = np.random.exponential(scale=1/(lambda_1*T), size=1)[0]
        Y.append(Y_new + Y[-1])
    Y = Y[:-1]
    V = [V0]
    Z = np.random.randn(n+1)
    count = len(Y)
    V_hn = 0
    V_hp = 0
    V_next = 0
    for j in range(1, n + 1):
        if count != 0 and dt * (j - 1) <= Y[len(Y) - count] and dt * j > Y[len(Y) - count]:
            V_hn = V[-1] * (1 + mu * (Y[len(Y) - count] - dt * (j - 1)) + sigma * np.sqrt((Y[len(Y) - count] - dt * (j - 1))) * Z[j - 1])
            V_hp = V_hn * (1 + gamma)
            V_next = V_hp * (1 + mu * (dt * j - Y[len(Y) - count]) + sigma * np.sqrt((dt * j - Y[len(Y) - count])) * Z[j])
            V.append(V_next)
            count -= 1
        else:
            V.append(V[-1] * (1 + mu * dt + sigma * np.sqrt(dt) * Z[j]))

    Q = np.where(V <= qt * Lt)[0]*dt

    if len(Q) != 0:
        Q = Q[0]
    else:
        Q = T+1

    X = [np.random.exponential(scale=1/(lambda_2*T), size=1)[0]]
    while X[-1] < T:
        X_new = np.random.exponential(scale=1/(lambda_2*T), size=1)[0]
        X.append(X_new + X[-1])
    X = X[:-1]
    V_q = [V0]
    
    count_1 = len(X)
    if count_1 != 0:
        S = X[0]
    else:
        S = T+1

    V_hn = 0
    V_hp = 0
    V_next = 0

    for k in range(1, n + 1):
        if count_1 != 0 and dt * (k - 1) <= X[len(X) - count_1] and dt * k > X[len(X) - count_1]:
            V_hn = V_q[-1] * (1 + mu * (X[len(X) - count_1] - dt * (k - 1)) + sigma * np.sqrt((X[len(X) - count_1] - dt * (k - 1))) * Z[k - 1])
            V_hp = V_hn * (1 + gamma)
            V_next = V_hp * (1 + mu * (dt * k - X[len(X) - count_1]) + sigma * np.sqrt((dt * k - X[len(X) - count_1])) * Z[k])
            V_q.append(V_next)
            count_1 -= 1
        else:
            V_q.append(V_q[-1] * (1 + mu * dt + sigma * np.sqrt(dt) * Z[k]))


    tao = np.minimum(Q,S)
    payoff = 0
    if tao >=  T:
        payoff = 0

    elif tao < T:
        if Q < S:
            position = int(tao/dt)
            Lq = a - b * c **(12 * Q)
            payoff = np.exp(-r0 * tao)*np.max(Lq - epsilon * V_q[position],0)

        if Q>=S:
            position = int(tao/dt)
            Ls = a - b * c **(12 * S)
            payoff = np.exp(-r0 * tao)*np.abs(Ls - epsilon * V[position])


    payoff_final[i] = payoff
    tao_list[i] = tao
    
value = payoff_final.mean()
prob = sum(payoff_final != 0)/len(payoff_final)
exp_tao = tao_list[np.where(tao_list < T)].mean()
result = [value, prob, exp_tao]
result


[2792.8406901048547, 0.9999, 0.3388106168141614]